In [ ]:
import random
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.backend import reshape
from keras.utils.np_utils import to_categorical
import os


import tensorflow as tf


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = tf.keras.models.load_model('/saved_model/tictactoe_model')

# Check its architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               2000      
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 125)               25125     
_________________________________________________________________
dense_2 (Dense)              (None, 75)                9450      
_________________________________________________________________
dropout_1 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 25)                1900      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 7

In [ ]:
# Get an empty board
#
# 0 indicates an empty space, 1 indicates an 'X' (player 1), and 2 indicates an 'O' (player 2)
#
# Initially the board is empty, so we return a 3x3 array of zeroes.
def initBoard():
    board = [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]
    return board

# Print the current state of the board
def printBoard(board):
    for i in range(len(board)):
        for j in range(len(board[i])):
            mark = ' '
            if board[i][j] == 1:
                mark = 'X'
            elif board[i][j] == 2:
                mark = 'O'
            if (j == len(board[i]) - 1):
                print(mark)
            else:
                print(str(mark) + "|", end='')
        if (i < len(board) - 1):
            print("-----")

In [ ]:
# Get a list of valid moves (indices into the board)
def getMoves(board):
    moves = []
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 0:
                moves.append((i, j))
    return moves



In [ ]:
# Declare a winner
#
# -1 = game not over
#  0 = draw
#  1 = 'X' wins (player 1)
#  2 = 'O' wins (player 2)
def getWinner(board):
    candidate = 0
    won = 0
    
    # Check rows
    for i in range(len(board)):
        candidate = 0
        for j in range(len(board[i])):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif j == len(board[i]) - 1:
                won = candidate
    
    if won > 0:
        return won

    # Check columns
    for j in range(len(board[0])):
        candidate = 0
        for i in range(len(board)):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif i == len(board) - 1:
                won = candidate
    
    if won > 0:
        return won
    
    # Check diagonals
    candidate = 0
    for i in range(len(board)):
        if board[i][i] == 0:
            break
        if candidate == 0:
            candidate = board[i][i]
        if candidate != board[i][i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    candidate = 0
    for i in range(len(board)):
        if board[2 - i][2 - i] == 0:
            break
        if candidate == 0:
            candidate = board[2 - i][2 - i]
        if candidate != board[2 - i][2 - i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    # Still no winner?
    if (len(getMoves(board)) == 0):
        # It's a draw
        return 0
    else:
        # Still more moves to make
        return -1

In [ ]:
random.seed()

# Get best next move for the given player at the given board position
def bestMove(board, model, player, rnd=0):
    scores = []
    moves = getMoves(board)
    
    # Make predictions for each possible move
    for i in range(len(moves)):
        future = np.array(board)
        future[moves[i][0]][moves[i][1]] = player
        prediction = model.predict(future.reshape((-1, 9)))[0]
        if player == 1:
            winPrediction = prediction[1]
            lossPrediction = prediction[2]
        else:
            winPrediction = prediction[2]
            lossPrediction = prediction[1]
        drawPrediction = prediction[0]
        if winPrediction - lossPrediction > 0:
            scores.append(winPrediction - lossPrediction)
        else:
            scores.append(drawPrediction - lossPrediction)

    # Choose the best move with a random factor
    bestMoves = np.flip(np.argsort(scores))
    for i in range(len(bestMoves)):
        if random.random() * rnd < 0.5:
            return moves[bestMoves[i]]

    # Choose a move completely at random
    return moves[random.randint(0, len(moves) - 1)]


In [ ]:
def board_pos(x):
  if x == 1:
    return (0,0)
  elif x == 2:
    return (0,1)
  elif x == 3:
    return (0,2)
  elif x == 4:
    return (1,0)
  elif x == 5:
    return (1,1)
  elif x == 6 :
    return (1,2)
  elif x == 7:
    return (2,0)
  elif x == 8:
    return (2,1)
  elif x == 9:
    return (2,2)

In [ ]:
h = board_pos(2)
a,b = h
print(a)
print(b)

0
1


In [ ]:
#  play between model and human
def model_human():

  m=0
  n=0
  i = 0
  board = initBoard()
  while getWinner(board) == -1:
    if i%2 == 0:
        move = bestMove(board, model, 1)
        board[move[0]][move[1]] = 1
        print('\n Machine Move \n')
        printBoard(board)
        print('\n')
        i = i+1



    else:
      
      
      pos = int(input("\n Which empty position you want to play ? : \n ( 1 to 9 )starting, 1 as upper left position and 9 as lower right position \n)"))
      m,n= board_pos(pos)
      board[m][n] = 2
      print('\n Human Move \n')
      printBoard(board)
      print('\n')
      i=i+1

  
    if getWinner(board) == 1:
      print('\n *** Game Over *** \n Machine won!')
    elif getWinner(board)== 2:
      print('\n *** Game Over *** \n Human Won!')
    elif getWinner(board)==0:
      print('\n *** Game Over *** \n Game Drawn!')

        

    


  


In [ ]:
model_human()

NameError: ignored